In [1]:
import sqlite3 
import pandas as pd

In [2]:
conn = sqlite3.connect("./merged_dataset_with_dates 2.sqlite")

In [3]:
def find_percent_num_crash_diff(counties):
    results = []
    for county in counties:
        county = county.upper()

        sql = "SELECT `County`, `Crash Year`, COUNT(distinct `Crash ID`) AS `Num Crashes` FROM crash_data WHERE `County` = ? GROUP BY `Crash Year`"
        df1 = pd.read_sql_query(sql, conn, params=[county])

        crashes = df1["Num Crashes"].tolist()
        num_crashes_2019 = crashes[0]
        num_crashes_2020 = crashes[1]
        diff = num_crashes_2019 - num_crashes_2020
        
        percent_diff = (diff/num_crashes_2019) * 100
        results.append(percent_diff)
        
    data = {'County':counties, '% Less Crashes':results}
    df2 = pd.DataFrame(data)
        
    return df2.sort_values('% Less Crashes', ascending=False)

### Difference in crashes of top 10 counties between 2019 and 2020

In [4]:
find_percent_num_crash_diff(['Harris','Dallas','Tarrant','Bexar','Travis','Collin','Denton','Hidalgo','El Paso'])

,County,% Less Crashes
5,Collin,46.593215
6,Denton,45.304878
4,Travis,40.489828
8,El Paso,40.071455
3,Bexar,39.933155
7,Hidalgo,38.333333
2,Tarrant,38.211832
0,Harris,33.774750
1,Dallas,31.551818


In [5]:
def compare_dd_crashes():
    results = []
    
    dd_sql = "SELECT `Crash Year`, COUNT(`Person Alcohol Result`) AS `dd_crashes` FROM crash_data WHERE `Person Alcohol Result` = '1 - POSITIVE' GROUP BY `Crash Year`"
    df1 = pd.read_sql_query(dd_sql, conn)
    
    dd_crashes = df1["dd_crashes"].tolist()
    dd_crashes_2019 = dd_crashes[0]
    dd_crashes_2020 = dd_crashes[1]
    
    crash_total_sql ="SELECT `Crash Year`, COUNT(distinct `Crash ID`) AS `Num Crashes` FROM crash_data GROUP BY `Crash Year`"
    df2 = pd.read_sql_query(crash_total_sql, conn)
    crashes = df2["Num Crashes"].tolist()
    crashes_2019 = crashes[0]
    crashes_2020 = crashes[1]
    
    results.append(dd_crashes_2019 / crashes_2019 * 100)
    results.append(dd_crashes_2020 / crashes_2020 * 100)
    
    df1["% of Total Crashes"] = results
   
    return df1

### Percent of Total Crashes that were Alcohol Related

In [6]:
compare_dd_crashes()

,Crash Year,dd_crashes,% of Total Crashes
0,2019,2596,1.696921
1,2020,1350,1.368558
